# Weapon/win corr last WM

Correlation between main primary weapon used and victory in round.

## Example correlation coefficient values
- correlation coefficient 0.06 of Deagles_x1 over 100 rounds where it was tagged could mean that
    - 53 were won by that team
    - 47 were lost
- correlation coefficient -0.04 of Deagles_x1 over 100 rounds:
    - 48 were won by that team
    - 52 were lost
- correlation coefficient 0.00 of Deagles_x1 over 100 rounds:
    - 50 were won by that team
    - 50 were lost

## How is it calculated
- each round's kill log is checked
- if one player used a weapon for more than 50% of his primary-weapon-kills then tag for that team is created, eg `SteyrAUG_x1`
- if there are two players in the team using mainly the same primary weapon in that round, that "team-round" will be tagged with `SteyrAUG_x2` instead of two `SteyrAUG_x1`
- win or lose tag is added for each "team-round"
- entries are roughly like this for each round:
    - "game 123123, round 0, team Red, tags: SteyrAug_x2, Deagles_x1, win"
    - "game 123123, round 0, team Blue, tags: M79_x1, MP5_x1, lose"
- rounds that ended with Tie are ignored altogether
- data is filtered to only include balanced games from correct playlist (see "Parameters" section)
- finally correlation is calculated between win tag and all other tags using [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)


## Parameters

In [3]:
from s2_analytics.constants import WEAPON_MODS_DATES

CHART_MAX_WIDTH = 10
CHART_HEIGHT_PER_ENTRY = 0.3

MINIMUM_SAMPLES = 30
MAX_IMBALANCE = 0.20
START_DATE = WEAPON_MODS_DATES[0]
PLAYLISTS = ["CTF-Standard-6"]

print(f"Start date: {START_DATE}")
print(f"Playlists: {','.join(PLAYLISTS)}")
print()
print(f"Min samples: {MINIMUM_SAMPLES}")
print(f"Max match imbalance: {MAX_IMBALANCE * 100:.0f}% \n"
      f"    (eg worst balance: Red team win probability {(50 - MAX_IMBALANCE * 100/2):.1f}% and {(50 + MAX_IMBALANCE * 100/2):.1f}% for Blue)")


Start date: 2022-12-28 00:00:00
Playlists: CTF-Standard-6

Min samples: 30
Max match imbalance: 20% 
    (eg worst balance: Red team win probability 40.0% and 60.0% for Blue)


In [ ]:
from pandas import DataFrame

from s2_analytics.filters import max_imbalance
import sqlite3

import pandas as pd
import seaborn as sns

from s2_analytics.collector.sqlite_collector import SqliteCollector
from s2_analytics.constants import WEAPONS_PRIMARY
from s2_analytics.importer import import_games
from s2_analytics.main_weapon_analyzer import MainWeaponRoundTagger
from s2_analytics.team_round_tag_correlation_analyzer import TeamRoundTagCorrelationAnalyzer
from s2_analytics.test_summary_collector import SummaryCollector

conn = sqlite3.connect("file::memory:")
sqlite_collector = SqliteCollector(sqlite_conn=conn).init()
summary_collector = SummaryCollector(conn, sqlite_collector)
tag_correlation_analyzer = TeamRoundTagCorrelationAnalyzer(conn, sqlite_collector, [
    MainWeaponRoundTagger([WEAPONS_PRIMARY])]).init()
import_games(
    "logs_ranked/",
    start_date=START_DATE,
    processors=[tag_correlation_analyzer, sqlite_collector],
    game_filters=[
        lambda g: g.playlist_code in ["CTF-Standard-6"],
        max_imbalance(MAX_IMBALANCE)
    ]
)
pass

## Data summary

In [ ]:
import tabulate

summary = summary_collector.get_summary().to_table()
tabulate.tabulate(summary, tablefmt='html')

## Aggregated result

In [ ]:
from matplotlib import pyplot as plt
from s2_analytics.test_main_weapon_analyzer import NO_RESULT_TAG_FILTER

correlation_per_map = tag_correlation_analyzer.calculate_win_correlation()

correlation_df = pd.DataFrame(correlation_per_map.items(), columns=["tag", "corr"])
counts_df = pd.DataFrame(
    tag_correlation_analyzer.tag_counts(NO_RESULT_TAG_FILTER).items(),
    columns=["tag", "cnt"])

df = pd.merge(correlation_df, counts_df, on=["tag"])


def filter_and_sort_for_corr_charts(df: DataFrame, min_samples, limit):
    df["corr_abs"] = df['corr'].abs()
    df = df[df.apply(lambda x: x["cnt"] > min_samples, axis=1)]
    return df.sort_values("corr_abs", ascending=False) \
        .head(n=limit) \
        .sort_values("corr", ascending=False)


df = filter_and_sort_for_corr_charts(df, MINIMUM_SAMPLES, 9999)

fig, axes = plt.subplots(1, 2, figsize=(CHART_MAX_WIDTH, len(df["corr"]) * CHART_HEIGHT_PER_ENTRY), sharey=True)
sns.barplot(x=df["corr"], y=df["tag"], ax=axes[0]) \
    .set(xlabel="Round victory correlation coefficient", ylabel="Main weapon tags")
sns.barplot(x=df["cnt"], y=df["tag"], ax=axes[1]) \
    .set(xlabel="Count of entries", ylabel="Main weapon tags")
pass

In [ ]:
## Each map separately
# from math import ceil
#
# correlation_per_map = tag_correlation_analyzer.calculate_win_correlation_per_map()
# counts_per_map = tag_correlation_analyzer.tag_counts_per_map(NO_RESULT_TAG_FILTER)
#
#
# def correlation_for_map(correlations: DataFrame, counts: DataFrame):
#     df = pd.merge(
#         on=["tag"],
#         left=pd.DataFrame(correlations.items(), columns=["tag", "corr"]),
#         right=pd.DataFrame(counts.items(), columns=["tag", "cnt"]))
#     return df
#
#
# max_corr = 0
# min_corr = 0
# total_entries = 0
# charts = []
# not_enough_data = []
# for map in sorted(list(counts_per_map.keys())):
#     correlations = correlation_per_map[map]
#     counts = counts_per_map[map]
#     df1 = pd.merge(on=["tag"],
#                    left=pd.DataFrame(correlations.items(), columns=["tag", "corr"]),
#                    right=pd.DataFrame(counts.items(), columns=["tag", "cnt"]))
#     df = df1
#     df = filter_and_sort_for_corr_charts(df, min_samples=(MINIMUM_SAMPLES), limit=5)
#     entries_count = len(df["corr"])
#     total_entries += entries_count
#     if entries_count > 0:
#         charts.append((map, df))
#         df_max = df["corr"].max()
#         if df_max > max_corr:
#             max_corr = df_max
#         df_min = df["corr"].min()
#         if df_min < min_corr:
#             min_corr = df_min
#     else:
#         not_enough_data.append(map)
#
# nrows = ceil(len(charts) / 2)
# fig, axes = plt.subplots(nrows, 2, figsize=(CHART_MAX_WIDTH, total_entries / len(charts) * nrows * 0.6),
#                          tight_layout=True)
# axes = axes.flatten()
# print(f"Not enough samples for {len(not_enough_data)} maps")
#
# for i, chart_data in enumerate(charts):
#     map, df = chart_data
#     axes[i].set(xlabel=None, ylabel=None)
#     axes[i].set(xlim=(min_corr, max_corr))
#     sns.barplot(x=df["corr"], y=df["tag"], ax=axes[i]) \
#         .set(title=map, xlabel=None, ylabel=None)

# pass